In [1]:
import keras
import numpy as np

/Users/perceval/anaconda/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/perceval/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [10]:
np.set_printoptions(linewidth=80, precision=4)

In [209]:
DISCUSSIONS_COUNT = 10
DISCUSSION_LENGTH = 32
TARGETS_COUNT = 16
EMBEDDING_DIM = 32,
LSTM_UNITS_COUNT = 32,
DROPOUT = 0.2

In [210]:
targets_pairs = np.array([np.random.choice(TARGETS_COUNT, 2, replace=False) for _ in range(DISCUSSIONS_COUNT)])

In [211]:
input_matrix = np.zeros((DISCUSSIONS_COUNT, DISCUSSION_LENGTH, TARGETS_COUNT))

# get a hint in the first utterance about the first speaker
input_matrix[np.arange(DISCUSSIONS_COUNT), 0, targets_pairs[:, 0]] = 1
input_matrix[np.arange(DISCUSSIONS_COUNT), 1, targets_pairs[:, 1]] = 1

In [212]:
output_matrix = np.zeros((DISCUSSIONS_COUNT, DISCUSSION_LENGTH, TARGETS_COUNT))
for i in range(DISCUSSION_LENGTH//2):
    output_matrix[np.arange(DISCUSSIONS_COUNT), i*2, targets_pairs[:, 0]] = 1
    output_matrix[np.arange(DISCUSSIONS_COUNT), i*2+1, targets_pairs[:, 1]] = 1

In [213]:
output_matrix[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,

In [217]:
input_layer = keras.layers.Input(shape=(None, TARGETS_COUNT)) # leave the dataset length that will be batched

people_hint_bag_layer = keras.layers.Dense(params["EMBEDDING_DIM"], activation='relu')

lstm_input = people_hint_bag_layer(input_layer)#keras.layers.Activation('relu')(concat_input)

# not mandatory to set the LSTM dim output to the people hints dim input but it seems more coherent

lstm_layer = keras.layers.GRU(params['LSTM_UNITS_COUNT'], return_sequences=True)
lstm_output = lstm_layer(lstm_input)

output_layer = keras.layers.Dense(TARGETS_COUNT, activation='softmax')
#lstm_layer = keras.layers.Dense(len(people)+1, activation='softmax')
output = output_layer(lstm_output)

In [218]:
model = keras.Model(inputs = input_layer, outputs = [output])
model.compile(loss='categorical_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])

In [219]:
# WITH EMBEDDING_DIM=32, LSTM = 32, N=16

for i in range(5):
    model.fit(input_matrix, output_matrix, epochs=1000, verbose=0)
    print(model.evaluate(input_matrix, output_matrix))

10/10 [==============================] - 1s 139ms/step
[0.8963462710380554, 0.5218750238418579]
10/10 [==============================] - 0s 860us/step
[0.26403096318244934, 0.918749988079071]
10/10 [==============================] - 0s 654us/step
[0.0444922149181366, 1.0]
10/10 [==============================] - 0s 670us/step
[0.01800515502691269, 1.0]
10/10 [==============================] - 0s 802us/step
[0.008890574797987938, 1.0]


In [207]:
# WITH EMBEDDING_DIM=32, LSTM = 32, N=8

for i in range(5):
    model.fit(input_matrix, output_matrix, epochs=2000, verbose=0)
    print(model.evaluate(input_matrix, output_matrix))

10/10 [==============================] - 1s 112ms/step
[0.6733682751655579, 0.53125]
10/10 [==============================] - 0s 767us/step
[0.3160506784915924, 0.831250011920929]
10/10 [==============================] - 0s 1ms/step
[0.1443367600440979, 0.8999999761581421]
10/10 [==============================] - 0s 689us/step
[0.13329936563968658, 0.90625]
10/10 [==============================] - 0s 723us/step
[0.004640371073037386, 1.0]


In [202]:
# WITH LSTM=64, N=8

for i in range(5):
    model.fit(input_matrix, output_matrix, epochs=2000, verbose=0)
    print(model.evaluate(input_matrix, output_matrix))

10/10 [==============================] - 1s 125ms/step
[0.7263706922531128, 0.5249999761581421]
10/10 [==============================] - 0s 770us/step
[0.6546169519424438, 0.5625]
10/10 [==============================] - 0s 661us/step
[0.6356932520866394, 0.546875]
10/10 [==============================] - 0s 681us/step
[0.5890239477157593, 0.6343749761581421]
10/10 [==============================] - 0s 1ms/step
[0.4870223104953766, 0.6625000238418579]


In [193]:
# WITH LSTM=32, N=8

for i in range(5):
    model.fit(input_matrix, output_matrix, ecpochs=2000, verbose=0)
    print(model.evaluate(input_matrix, output_matrix))

10/10 [==============================] - 1s 108ms/step
[0.6815983057022095, 0.515625]
10/10 [==============================] - 0s 671us/step
[0.31143176555633545, 0.796875]
10/10 [==============================] - 0s 704us/step
[0.1881525218486786, 0.875]
10/10 [==============================] - 0s 665us/step
[0.12290861457586288, 0.9375]
10/10 [==============================] - 0s 617us/step
[0.06663595139980316, 0.9593750238418579]


In [182]:
model.fit(input_matrix, output_matrix, epochs=10, verbose=1)

Epoch 1/10
10/10 [==============================] - 0s 5ms/step - loss: 0.6507 - acc: 0.5531
Epoch 2/10
10/10 [==============================] - 0s 5ms/step - loss: 0.6507 - acc: 0.5531
Epoch 3/10
10/10 [==============================] - 0s 4ms/step - loss: 0.6507 - acc: 0.5531
Epoch 4/10
10/10 [==============================] - 0s 5ms/step - loss: 0.6507 - acc: 0.5531
Epoch 5/10
10/10 [==============================] - 0s 4ms/step - loss: 0.6507 - acc: 0.5531
Epoch 6/10
10/10 [==============================] - 0s 4ms/step - loss: 0.6507 - acc: 0.5531
Epoch 7/10
10/10 [==============================] - 0s 5ms/step - loss: 0.6506 - acc: 0.5531
Epoch 8/10
10/10 [==============================] - 0s 5ms/step - loss: 0.6506 - acc: 0.5531
Epoch 9/10
10/10 [==============================] - 0s 6ms/step - loss: 0.6506 - acc: 0.5531
Epoch 10/10
10/10 [==============================] - 0s 4ms/step - loss: 0.6506 - acc: 0.5531


In [183]:
model.fit(input_matrix, output_matrix, epochs=2000, verbose=0)

In [184]:
model.fit(input_matrix, output_matrix, epochs=10, verbose=1)

Epoch 1/10
10/10 [==============================] - 0s 4ms/step - loss: 0.2569 - acc: 0.8531
Epoch 2/10
10/10 [==============================] - 0s 3ms/step - loss: 0.2568 - acc: 0.8500
Epoch 3/10
10/10 [==============================] - 0s 3ms/step - loss: 0.2567 - acc: 0.8500
Epoch 4/10
10/10 [==============================] - 0s 3ms/step - loss: 0.2566 - acc: 0.8531
Epoch 5/10
10/10 [==============================] - 0s 3ms/step - loss: 0.2564 - acc: 0.8531
Epoch 6/10
10/10 [==============================] - 0s 3ms/step - loss: 0.2563 - acc: 0.8531
Epoch 7/10
10/10 [==============================] - 0s 4ms/step - loss: 0.2562 - acc: 0.8531
Epoch 8/10
10/10 [==============================] - 0s 3ms/step - loss: 0.2561 - acc: 0.8531
Epoch 9/10
10/10 [==============================] - 0s 3ms/step - loss: 0.2559 - acc: 0.8500
Epoch 10/10
10/10 [==============================] - 0s 4ms/step - loss: 0.2558 - acc: 0.8500


In [185]:
model.fit(input_matrix, output_matrix, epochs=2000, verbose=0)

In [186]:
model.fit(input_matrix, output_matrix, epochs=10, verbose=1)

Epoch 1/10
10/10 [==============================] - 0s 4ms/step - loss: 0.0047 - acc: 1.0000
Epoch 2/10
10/10 [==============================] - 0s 3ms/step - loss: 0.0047 - acc: 1.0000
Epoch 3/10
10/10 [==============================] - 0s 4ms/step - loss: 0.0047 - acc: 1.0000
Epoch 4/10
10/10 [==============================] - 0s 3ms/step - loss: 0.0047 - acc: 1.0000
Epoch 5/10
10/10 [==============================] - 0s 3ms/step - loss: 0.0047 - acc: 1.0000
Epoch 6/10
10/10 [==============================] - 0s 4ms/step - loss: 0.0046 - acc: 1.0000
Epoch 7/10
10/10 [==============================] - 0s 3ms/step - loss: 0.0046 - acc: 1.0000
Epoch 8/10
10/10 [==============================] - 0s 4ms/step - loss: 0.0046 - acc: 1.0000
Epoch 9/10
10/10 [==============================] - 0s 3ms/step - loss: 0.0046 - acc: 1.0000
Epoch 10/10
10/10 [==============================] - 0s 3ms/step - loss: 0.0046 - acc: 1.0000


In [162]:
input_matrix[0]

array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [222]:
model.predict(np.array([[
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]]))

array([[[7.9148e-07, 1.2005e-03, 6.7917e-07, ..., 2.1546e-05, 2.2659e-06,
         3.2168e-08],
        [1.7155e-05, 1.7705e-03, 1.0621e-05, ..., 1.2168e-05, 4.3264e-06,
         2.2561e-04],
        [4.4368e-06, 4.4145e-04, 4.6635e-06, ..., 1.8069e-08, 2.8505e-06,
         3.3394e-03],
        ...,
        [2.1087e-05, 1.8694e-03, 1.2891e-05, ..., 1.4140e-05, 5.0801e-06,
         2.7778e-04],
        [4.8591e-06, 4.3603e-04, 5.1810e-06, ..., 2.2618e-08, 3.0554e-06,
         3.2629e-03],
        [2.1087e-05, 1.8694e-03, 1.2891e-05, ..., 1.4140e-05, 5.0801e-06,
         2.7778e-04]]], dtype=float32)

In [223]:
_[0][-1]

array([2.1087e-05, 1.8694e-03, 1.2891e-05, 5.8751e-04, 9.8956e-01, 6.5572e-06,
       1.0620e-06, 6.7972e-03, 1.9439e-06, 7.1675e-04, 1.2168e-04, 5.9335e-09,
       1.1342e-05, 1.4140e-05, 5.0801e-06, 2.7778e-04], dtype=float32)